In [ ]:
import sys
import math
import numpy as np
import serial
import serial.tools.list_ports
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, 
                             QHBoxLayout, QLabel, QLineEdit, QPushButton, 
                             QGraphicsScene, QGraphicsView, QGraphicsEllipseItem,
                             QGraphicsLineItem, QMessageBox, QGroupBox, QComboBox)
from PyQt5.QtGui import QPen, QBrush, QColor, QPainter
from PyQt5.QtCore import Qt, QTimer, pyqtSignal, QThread

from PyQt5.QtGui import QFont
from PyQt5.QtWidgets import QGraphicsTextItem



# ---------------------- Global Scaling Factor (Total 15x Zoom) ----------------------
BASE_SCALE = 10       # Base zoom factor
EXTRA_SCALE = 1.5     # Additional 1.5x zoom
SCALE_FACTOR = BASE_SCALE * EXTRA_SCALE  # Total zoom factor = 15

# ---------------------- Utility Function: Shortest Angle Path Calculation ----------------------
def normalize_angle(angle_deg):
    """Normalize angle to -180 ~ 180° range"""
    angle_deg = math.fmod(angle_deg, 360.0)
    if angle_deg > 180.0:
        angle_deg -= 360.0
    elif angle_deg < -180.0:
        angle_deg += 360.0
    return angle_deg

def shortest_angle_diff(target_deg, current_deg):
    """Calculate shortest rotation angle from current to target (-180~180°)"""
    diff = normalize_angle(target_deg - current_deg)
    return diff

def get_shortest_target_angle(current_deg, target_deg):
    """Get target angle with shortest path (based on current angle)"""
    diff = shortest_angle_diff(target_deg, current_deg)
    return current_deg + diff

# ---------------------- Serial Communication Thread (Adapt to Small Motor Reverse) ----------------------
class SerialThread(QThread):
    # Define signals: receive Arduino data, serial status change
    recv_signal = pyqtSignal(float, float, float, float)  # m1 current, m1 target, m2 current, m2 target
    status_signal = pyqtSignal(str)

    def __init__(self, port, baudrate=9600):
        super().__init__()
        self.port = port
        self.baudrate = baudrate
        self.ser = None
        self.is_running = True
        self.target_angles = (0.0, 0.0)  # Target angles to send (m1, m2)
        self.current_angles = (0.0, 0.0) # Current angles (for shortest path calculation)
        self.need_send = False

    def run(self):
        try:
            self.ser = serial.Serial(self.port, self.baudrate, timeout=0.1)
            self.status_signal.emit(f"Serial port connected: {self.port}")
            while self.is_running:
                # 1. Send target angles (small motor angle reversed + shortest path)
                if self.need_send:
                    # Calculate shortest path target angles based on current angles
                    m1_short = get_shortest_target_angle(self.current_angles[0], self.target_angles[0])
                    m2_short = get_shortest_target_angle(self.current_angles[1], self.target_angles[1])
                    
                    # Reverse small motor target angle to adapt downward shaft
                    send_m1 = m1_short
                    send_m2 = -m2_short  # Reverse small motor angle
                    
                    send_data = f"{send_m1},{send_m2}\n"
                    self.ser.write(send_data.encode('utf-8'))
                    self.need_send = False
                    self.status_signal.emit(f"Shortest path sent: m1={m1_short:.1f}°(orig{self.target_angles[0]:.1f}°), m2={m2_short:.1f}°(orig{self.target_angles[1]:.1f}°)")
                
                # 2. Receive Arduino feedback data (small motor angle reversed)
                if self.ser.in_waiting > 0:
                    try:
                        line = self.ser.readline().decode('utf-8').strip()
                        if line:
                            # Parse Arduino output format: m1 current m1 target m2 current m2 target
                            parts = line.split()
                            if len(parts) == 4:
                                m1_current = float(parts[0])
                                m1_target = float(parts[1])
                                # Key modification: reverse Arduino feedback small motor angle to restore display logic
                                m2_current = -float(parts[2])  # Reverse small motor current angle
                                m2_target = -float(parts[3])   # Reverse small motor target angle
                                
                                # Update current angles (for next shortest path calculation)
                                self.current_angles = (m1_current, m2_current)
                                self.recv_signal.emit(m1_current, m1_target, m2_current, m2_target)
                    except Exception as e:
                        self.status_signal.emit(f"Data parsing error: {str(e)}")
        except Exception as e:
            self.status_signal.emit(f"Serial port error: {str(e)}")
        finally:
            if self.ser and self.ser.is_open:
                self.ser.close()
            self.status_signal.emit("Serial port disconnected")

    def send_target(self, m1_angle, m2_angle):
        self.target_angles = (m1_angle, m2_angle)
        self.need_send = True

    def stop(self):
        """Stop thread"""
        self.is_running = False
        self.wait()

# ---------------------- Robotic Arm Control Main Window (Restore Coordinate Numbers) ----------------------
class TwoLinkArmController(QMainWindow):
    def __init__(self):
        super().__init__()
        # Robotic arm core parameters (two 8cm links, total 15x zoom for display)
        self.l1 = 8.0 * SCALE_FACTOR  
        self.l2 = 8.0 * SCALE_FACTOR  
        
        # Joint angles (radians) - updated in real-time
        self.theta1 = 0.0  
        self.theta2 = 0.0  
        
        # End effector position (scaled coordinates) - updated in real-time
        self.end_x = 0.0
        self.end_y = 0.0
        
        # Serial port related
        self.serial_thread = None
        self.current_m1 = 0.0  # Arduino feedback m1 current angle
        self.current_m2 = 0.0  # Arduino feedback m2 current angle (reversed)
        self.target_m1 = 0.0   # Arduino m1 target angle
        self.target_m2 = 0.0   # Arduino m2 target angle (reversed)

        # Initialize UI
        self.init_ui()





        
        # Initial forward kinematics calculation
        self.forward_kinematics()
        # Initialize graphic display
        self.update_arm_display()

    def init_ui(self):
        """Initialize graphic interface (restore coordinate numbers)"""
        self.setWindowTitle('Two-Link Robotic Arm Controller (15x Coordinate Zoom + Scale Numbers)')
        self.setGeometry(100, 100, 1600, 1000)  # Main window size
        
        # Central widget
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QHBoxLayout(central_widget)
        main_layout.setSpacing(20)
        main_layout.setContentsMargins(20, 20, 20, 20)
        
        # ---------------------- Left Control Panel ----------------------
        control_panel = QWidget()
        control_panel.setFixedWidth(400)
        control_layout = QVBoxLayout(control_panel)
        
        # 1. Serial port settings group
        serial_group = QGroupBox('Serial Port Settings')
        serial_layout = QHBoxLayout(serial_group)
        
        self.port_combo = QComboBox()
        self.refresh_ports()
        serial_layout.addWidget(QLabel('Serial Port:'))
        serial_layout.addWidget(self.port_combo)
        
        refresh_btn = QPushButton('Refresh Ports')
        refresh_btn.clicked.connect(self.refresh_ports)
        serial_layout.addWidget(refresh_btn)
        
        self.connect_btn = QPushButton('Connect Serial Port')
        self.connect_btn.clicked.connect(self.toggle_serial)
        serial_layout.addWidget(self.connect_btn)
        
        # 2. Forward Kinematics control group
        fk_group = QGroupBox('Forward Kinematics (FK) - Angle → End Effector Position')
        fk_layout = QVBoxLayout(fk_group)
        
        angle1_layout = QHBoxLayout()
        angle1_layout.addWidget(QLabel('Joint 1 Angle (°):'))
        self.theta1_input = QLineEdit('0')
        self.theta1_input.setPlaceholderText('Enter 0-360° (supports negative)')
        angle1_layout.addWidget(self.theta1_input)
        
        angle2_layout = QHBoxLayout()
        angle2_layout.addWidget(QLabel('Joint 2 Angle (°):'))
        self.theta2_input = QLineEdit('0')
        self.theta2_input.setPlaceholderText('Enter -180-180° (supports negative)')
        angle2_layout.addWidget(self.theta2_input)
        
        self.calc_fk_btn = QPushButton('Calculate End Position (FK)')
        self.calc_fk_btn.clicked.connect(self.calculate_fk)
        fk_layout.addLayout(angle1_layout)
        fk_layout.addLayout(angle2_layout)
        fk_layout.addWidget(self.calc_fk_btn)
        
        self.fk_result_label = QLabel('End Effector Position: X=0.00 cm, Y=0.00 cm')
        fk_layout.addWidget(self.fk_result_label)
        
        # 3. Inverse Kinematics control group
        ik_group = QGroupBox('Inverse Kinematics (IK) - Position → Angle (supports negative coordinates)')
        ik_layout = QVBoxLayout(ik_group)
        
        x_layout = QHBoxLayout()
        x_layout.addWidget(QLabel('Target X (cm):'))
        self.target_x_input = QLineEdit()
        self.target_x_input.setPlaceholderText('Enter -16~16 cm (supports negative)')
        x_layout.addWidget(self.target_x_input)
        
        y_layout = QHBoxLayout()
        y_layout.addWidget(QLabel('Target Y (cm):'))
        self.target_y_input = QLineEdit()
        self.target_y_input.setPlaceholderText('Enter -16~16 cm (supports negative)')
        y_layout.addWidget(self.target_y_input)
        
        self.calc_ik_btn = QPushButton('Calculate Joint Angles (IK)')
        self.calc_ik_btn.clicked.connect(self.calculate_ik)
        ik_layout.addLayout(x_layout)
        ik_layout.addLayout(y_layout)
        ik_layout.addWidget(self.calc_ik_btn)
        
        self.ik_result_label = QLabel('Joint Angles: θ1=0.00°, θ2=0.00°')
        ik_layout.addWidget(self.ik_result_label)
        
        # 4. Motion control button group
        motion_group = QWidget()
        motion_layout = QHBoxLayout(motion_group)
        
        self.move_to_ik_btn = QPushButton('Move to IK Target')
        self.move_to_ik_btn.clicked.connect(self.move_to_ik_target)
        self.move_to_ik_btn.setEnabled(False)
        
        self.move_to_fk_btn = QPushButton('Move to FK Angles')
        self.move_to_fk_btn.clicked.connect(self.move_to_fk_angle)
        self.move_to_fk_btn.setEnabled(False)
        
        self.reset_btn = QPushButton('Reset Robotic Arm')
        self.reset_btn.clicked.connect(self.reset_arm)
        motion_layout.addWidget(self.move_to_ik_btn)
        motion_layout.addWidget(self.move_to_fk_btn)
        motion_layout.addWidget(self.reset_btn)
        
        # 5. Arduino real-time status group
        status_group = QGroupBox('Arduino Real-Time Status')
        status_layout = QVBoxLayout(status_group)
        
        self.m1_current_label = QLabel('Large Motor Current Angle: 0.00°')
        self.m1_target_label = QLabel('Large Motor Target Angle: 0.00°')
        self.m2_current_label = QLabel('Small Motor Current Angle: 0.00°')
        self.m2_target_label = QLabel('Small Motor Target Angle: 0.00°')
        self.shortest_path_label = QLabel('Shortest Path Hint: None')
        self.serial_status_label = QLabel('Serial Port Status: Disconnected')
        
        status_layout.addWidget(self.m1_current_label)
        status_layout.addWidget(self.m1_target_label)
        status_layout.addWidget(self.m2_current_label)
        status_layout.addWidget(self.m2_target_label)
        status_layout.addWidget(self.shortest_path_label)
        status_layout.addWidget(self.serial_status_label)


        # 6. Trajectory / multi-point motion
        
        
        # Assemble control panel
        control_layout.addWidget(serial_group)
        control_layout.addWidget(fk_group)
        control_layout.addWidget(ik_group)
        control_layout.addWidget(motion_group)
        control_layout.addWidget(status_group)
        control_layout.addStretch()
        
        # ---------------------- Right Simulation View (Restore Coordinate Numbers) ----------------------
        # 1. Create scene (canvas range 15x zoom: -300~300)
        self.scene = QGraphicsScene()
        self.scene.setSceneRect(-300, -300, 600, 600)  # Original -20~20 → 15x zoom -300~300
        
        # 2. Create view
        self.view = QGraphicsView(self.scene)
        self.view.setRenderHint(QPainter.Antialiasing, True)
        self.view.setMinimumSize(1100, 900)  # View size
        
        # Draw coordinate system (15x zoom)
        # X axis
        x_axis = QGraphicsLineItem(-300, 0, 300, 0)
        x_axis.setPen(QPen(QColor(150, 150, 150), 8))  # Thicker axis line
        self.scene.addItem(x_axis)
        # Y axis
        y_axis = QGraphicsLineItem(0, -300, 0, 300)
        y_axis.setPen(QPen(QColor(150, 150, 150), 8))  # Thicker axis line
        self.scene.addItem(y_axis)
        
        # Scale marks + numbers (every 30 units = original 2cm, 15x zoom total)
        for i in range(-240, 270, 30):
            # Calculate original cm value
            original_cm = i / SCALE_FACTOR
            
            # X axis scale marks
            x_tick = QGraphicsLineItem(i, -8, i, 8)
            x_tick.setPen(QPen(QColor(150, 150, 150), 5))
            self.scene.addItem(x_tick)
            # X axis scale text (force integer/one decimal place, white text more visible)


            font = QFont("Arial", 16)
            font.setBold(True)

            # X轴刻度文字
            x_text_item = QGraphicsTextItem(f"{original_cm:.0f}" if original_cm.is_integer() else f"{original_cm:.1f}")
            x_text_item.setFont(font)
            x_text_item.setDefaultTextColor(QColor(255, 255, 255))
            x_text_item.setFlag(QGraphicsTextItem.ItemIgnoresTransformations, True)  # 永远清晰，不随缩放翻转
            x_text_item.setPos(i - 10, -35)  # 放在轴下方（避免遮挡）
            self.scene.addItem(x_text_item)
            
            # Y轴刻度文字（避免原点叠字）
            if i != 0:
                y_text_item = QGraphicsTextItem(f"{original_cm:.0f}" if original_cm.is_integer() else f"{original_cm:.1f}")
                y_text_item.setFont(font)
                y_text_item.setDefaultTextColor(QColor(255, 255, 255))
                y_text_item.setFlag(QGraphicsTextItem.ItemIgnoresTransformations, True)
                y_text_item.setPos(15, i - 10)
                self.scene.addItem(y_text_item)
            # x_text = QLabel(f"{original_cm:.0f}" if original_cm.is_integer() else f"{original_cm:.1f}")
            # x_text.setStyleSheet("font-size: 28px; font-weight: bold; color: #fff; background: rgba(0,0,0,0.5); padding: 2px 8px; border-radius: 4px;")
            # x_text_item = self.scene.addWidget(x_text)
            # x_text_item.setPos(i - 20, 15)  # Adjust position to avoid occlusion
            font = QFont("Arial", 16)
            font.setBold(True)
            
            label = f"{original_cm:.0f}" if float(original_cm).is_integer() else f"{original_cm:.1f}"
            
            x_text_item = QGraphicsTextItem(label)
            x_text_item.setFont(font)
            x_text_item.setDefaultTextColor(QColor(255, 255, 255))
            x_text_item.setFlag(QGraphicsTextItem.ItemIgnoresTransformations, True)  # 不随缩放/翻转
            x_text_item.setPos(i - 10, -35)  # 放到x轴下方（避免挡住机械臂）
            self.scene.addItem(x_text_item)



            if i != 0:
                y_text_item = QGraphicsTextItem(label)
                y_text_item.setFont(font)
                y_text_item.setDefaultTextColor(QColor(255, 255, 255))
                y_text_item.setFlag(QGraphicsTextItem.ItemIgnoresTransformations, True)
                y_text_item.setPos(15, i - 10)
                self.scene.addItem(y_text_item)
            # Y axis scale marks
            # y_tick = QGraphicsLineItem(-8, i, 8, i)
            # y_tick.setPen(QPen(QColor(150, 150, 150), 5))
            # self.scene.addItem(y_tick)
            # # Y axis scale text (force integer/one decimal place, white text more visible)
            # y_text = QLabel(f"{original_cm:.0f}" if original_cm.is_integer() else f"{original_cm:.1f}")
            # y_text.setStyleSheet("font-size: 28px; font-weight: bold; color: #fff; background: rgba(0,0,0,0.5); padding: 2px 8px; border-radius: 4px;")
            # y_text_item = self.scene.addWidget(y_text)
            # y_text_item.setPos(15, i - 20)  # Adjust position to avoid occlusion
        
        # Draw robotic arm elements (15x zoom)
        # Base (joint 1)
        self.base = QGraphicsEllipseItem(-25, -25, 50, 50)
        self.base.setBrush(QBrush(QColor(0, 0, 0)))
        self.scene.addItem(self.base)
        
        # Link 1
        self.link1 = QGraphicsLineItem()
        self.link1.setPen(QPen(QColor(255, 0, 0), 12))  # Thicker link
        self.scene.addItem(self.link1)
        
        # Link 2
        self.link2 = QGraphicsLineItem()
        self.link2.setPen(QPen(QColor(0, 255, 0), 12))  # Thicker link
        self.scene.addItem(self.link2)
        
        # Joint 2 marker
        self.joint2 = QGraphicsEllipseItem(-20, -20, 40, 40)
        self.joint2.setBrush(QBrush(QColor(128, 128, 128)))
        self.scene.addItem(self.joint2)
        
        # End effector
        self.end_effector = QGraphicsEllipseItem(-20, -20, 40, 40)
        self.end_effector.setBrush(QBrush(QColor(0, 0, 255)))
        self.scene.addItem(self.end_effector)
        
        # IK target marker
        self.ik_target_marker = QGraphicsEllipseItem(-25, -25, 50, 50)
        self.ik_target_marker.setBrush(QBrush(QColor(255, 255, 0, 150)))
        self.ik_target_marker.setVisible(False)
        self.scene.addItem(self.ik_target_marker)


        
        



        
                
        # Assemble main layout
        main_layout.addWidget(control_panel)
        main_layout.addWidget(self.view)

    # ---------------------- Serial Port Related Methods ----------------------
    def refresh_ports(self):
  
        self.port_combo.clear()
        ports = serial.tools.list_ports.comports()
        for port in ports:
            self.port_combo.addItem(port.device)

    def toggle_serial(self):
        """Connect/Disconnect serial port"""
        if self.serial_thread and self.serial_thread.isRunning():
            # Disconnect serial port
            self.serial_thread.stop()
            self.serial_thread = None
            self.connect_btn.setText('Connect Serial Port')
            self.move_to_ik_btn.setEnabled(False)
            self.move_to_fk_btn.setEnabled(False)
            self.serial_status_label.setText('Serial Port Status: Disconnected')
        else:
            # Connect serial port
            if self.port_combo.count() == 0:
                QMessageBox.warning(self, 'Error', 'No available serial ports detected!')
                return
            port = self.port_combo.currentText()
            self.serial_thread = SerialThread(port, 9600)
            # Bind signals
            self.serial_thread.recv_signal.connect(self.update_arduino_data)
            self.serial_thread.status_signal.connect(self.update_serial_status)
            self.serial_thread.start()
            self.connect_btn.setText('Disconnect Serial Port')
            self.move_to_ik_btn.setEnabled(True)
            self.move_to_fk_btn.setEnabled(True)

    def update_arduino_data(self, m1_current, m1_target, m2_current, m2_target):
        """Update angle data from Arduino feedback"""
        self.current_m1 = m1_current
        self.current_m2 = m2_current
        self.target_m1 = m1_target
        self.target_m2 = m2_target
        
        # Update display
        self.m1_current_label.setText(f'Large Motor Current Angle: {m1_current:.2f}°')
        self.m1_target_label.setText(f'Large Motor Target Angle: {m1_target:.2f}°')
        self.m2_current_label.setText(f'Small Motor Current Angle: {m2_current:.2f}°')
        self.m2_target_label.setText(f'Small Motor Target Angle: {m2_target:.2f}°')
        
        # Calculate shortest path hint
        m1_diff = shortest_angle_diff(m1_target, m1_current)
        m2_diff = shortest_angle_diff(m2_target, m2_current)
        self.shortest_path_label.setText(f'Shortest Path Rotation: m1={m1_diff:.1f}°, m2={m2_diff:.1f}°')
        
        # Update simulation view
        self.theta1 = math.radians(m1_current)
        self.theta2 = math.radians(m2_current)
        self.forward_kinematics()
        self.update_arm_display()

    def update_serial_status(self, status):
        """Update serial port status display"""
        self.serial_status_label.setText(f'Serial Port Status: {status}')

    # ---------------------- Forward Kinematics Core Algorithm ----------------------
    def forward_kinematics(self, theta1=None, theta2=None):
        """Forward Kinematics: Angle → End Position (15x zoom coordinates)"""
        t1 = theta1 if theta1 is not None else self.theta1
        t2 = theta2 if theta2 is not None else self.theta2
        
        # Joint 2 position (15x zoom)
        joint2_x = self.l1 * math.cos(t1)
        joint2_y = self.l1 * math.sin(t1)
        
        # End effector position (15x zoom)
        end_x = joint2_x + self.l2 * math.cos(t1 + t2)
        end_y = joint2_y + self.l2 * math.sin(t1 + t2)
        
        # Update global variables
        if theta1 is not None and theta2 is not None:
            self.theta1 = t1
            self.theta2 = t2
        self.end_x = end_x
        self.end_y = end_y
        
        # Return original cm values (for display)
        return end_x/SCALE_FACTOR, end_y/SCALE_FACTOR

    # ---------------------- Inverse Kinematics Core Algorithm ----------------------
    def inverse_kinematics(self, target_x, target_y):
        """Inverse Kinematics: Positive/Negative Coordinates → Angles (input original cm values, internal 15x zoom calculation)"""
        # 15x zoom coordinates
        target_x_scaled = target_x * SCALE_FACTOR
        target_y_scaled = target_y * SCALE_FACTOR
        
        # Check workspace (zoomed)
        distance = math.hypot(target_x_scaled, target_y_scaled)
        if distance > self.l1 + self.l2 or distance < abs(self.l1 - self.l2):
            return 0.0, 0.0, False
        
        # Inverse solution calculation (zoomed coordinates)
        cos_theta2 = (target_x_scaled**2 + target_y_scaled**2 - self.l1**2 - self.l2**2) / (2 * self.l1 * self.l2)
        cos_theta2 = np.clip(cos_theta2, -1.0, 1.0)
        theta2 = math.acos(cos_theta2)
        
        alpha = math.atan2(target_y_scaled, target_x_scaled)
        beta = math.atan2(self.l2 * math.sin(theta2), self.l1 + self.l2 * math.cos(theta2))
        theta1 = alpha - beta
        
        # Normalize angles
        theta1 = normalize_angle(math.degrees(theta1))
        theta2 = normalize_angle(math.degrees(theta2))
        
        return math.radians(theta1), math.radians(theta2), True

    # ---------------------- Forward Kinematics Calculation Trigger ----------------------
    def calculate_fk(self):
        """Calculate FK: Angle → Position (display original cm values)"""
        try:
            theta1_deg = float(self.theta1_input.text().strip())
            theta2_deg = float(self.theta2_input.text().strip())
            
            # Normalize angles
            theta1_deg = normalize_angle(theta1_deg)
            theta2_deg = normalize_angle(theta2_deg)
            
            # Update input boxes
            self.theta1_input.setText(f"{theta1_deg:.1f}")
            self.theta2_input.setText(f"{theta2_deg:.1f}")
            
            theta1_rad = math.radians(theta1_deg)
            theta2_rad = math.radians(theta2_deg)
            
            # Execute FK calculation (return original cm values)
            end_x, end_y = self.forward_kinematics(theta1_rad, theta2_rad)
            
            # Update display
            self.fk_result_label.setText(f'End Effector Position: X={end_x:.2f} cm, Y={end_y:.2f} cm')
            
            # Update simulation view
            self.update_arm_display()
            
        except ValueError:
            QMessageBox.warning(self, 'Input Error', 'Please enter valid numeric angles!')

    # ---------------------- Inverse Kinematics Calculation Trigger ----------------------
    def calculate_ik(self):
        """Calculate IK: Positive/Negative Coordinates → Angles (input original cm values)"""
        try:
            target_x = float(self.target_x_input.text().strip())
            target_y = float(self.target_y_input.text().strip())
            
            # Coordinate range validation
            if not (-16 <= target_x <= 16) or not (-16 <= target_y <= 16):
                QMessageBox.warning(self, 'Input Error', 'Target coordinates must be in -16~16cm range!')
                return
            
            # Execute IK calculation
            theta1_rad, theta2_rad, valid = self.inverse_kinematics(target_x, target_y)
            
            if not valid:
                self.ik_result_label.setText('Joint Angles: Target point out of workspace!')
                QMessageBox.warning(self, 'Range Error', f'Target point ({target_x},{target_y}) is out of robotic arm workspace!')
                self.ik_target_marker.setVisible(False)
                return
            
            # Convert radians to degrees
            theta1_deg = math.degrees(theta1_rad)
            theta2_deg = math.degrees(theta2_rad)
            
            # Update display
            self.ik_result_label.setText(f'Joint Angles: θ1={theta1_deg:.2f}°, θ2={theta2_deg:.2f}°')
            
            # Save target angles
            self.ik_target_theta1 = theta1_deg
            self.ik_target_theta2 = theta2_deg
            
            # Show target marker (15x zoom)
            self.ik_target_marker.setPos(target_x*SCALE_FACTOR, target_y*SCALE_FACTOR)
            self.ik_target_marker.setVisible(True)
            
        except ValueError:
            QMessageBox.warning(self, 'Input Error', 'Please enter valid numeric coordinates!')

    # ---------------------- Motion Control ----------------------
    def move_to_ik_target(self):
        """Move to IK target point"""
        if not hasattr(self, 'ik_target_theta1'):
            QMessageBox.warning(self, 'Error', 'Please calculate IK angles first!')
            return
        if not self.serial_thread or not self.serial_thread.isRunning():
            QMessageBox.warning(self, 'Error', 'Please connect serial port first!')
            return
        
        # Send target angles
        self.serial_thread.send_target(self.ik_target_theta1, self.ik_target_theta2)

    def move_to_fk_angle(self):
        """Move to FK angles"""
        try:
            theta1_deg = float(self.theta1_input.text().strip())
            theta2_deg = float(self.theta2_input.text().strip())
            
            # Normalize angles
            theta1_deg = normalize_angle(theta1_deg)
            theta2_deg = normalize_angle(theta2_deg)
            
            if not self.serial_thread or not self.serial_thread.isRunning():
                QMessageBox.warning(self, 'Error', 'Please connect serial port first!')
                return
            
            # Send target angles
            self.serial_thread.send_target(theta1_deg, theta2_deg)
            
        except ValueError:
            QMessageBox.warning(self, 'Input Error', 'Please enter valid numeric angles!')

    # ---------------------- Graphic Update ----------------------
    def update_arm_display(self):
        """Update robotic arm simulation view (15x zoom coordinates)"""
        # Joint 2 position (zoomed)
        joint2_x = self.l1 * math.cos(self.theta1)
        joint2_y = self.l1 * math.sin(self.theta1)
        
        # Update links (zoomed coordinates)
        self.link1.setLine(0, 0, joint2_x, joint2_y)
        self.link2.setLine(joint2_x, joint2_y, self.end_x, self.end_y)
        
        # Update joint and end effector positions (center aligned, zoomed)
        self.joint2.setPos(joint2_x , joint2_y )
        self.end_effector.setPos(self.end_x , self.end_y )

    def reset_arm(self):
        """Reset robotic arm"""
        # Reset input boxes
        self.theta1_input.setText('0')
        self.theta2_input.setText('0')
        self.target_x_input.clear()
        self.target_y_input.clear()
        
        # Reset display
        self.fk_result_label.setText('End Effector Position: X=0.00 cm, Y=0.00 cm')
        self.ik_result_label.setText('Joint Angles: θ1=0.00°, θ2=0.00°')
        self.shortest_path_label.setText('Shortest Path Hint: None')
        self.ik_target_marker.setVisible(False)
        
        # Send reset angles
        if self.serial_thread and self.serial_thread.isRunning():
            self.serial_thread.send_target(0.0, 0.0)
        
        # Reset simulation view
        self.theta1 = 0.0
        self.theta2 = 0.0
        self.forward_kinematics()
        self.update_arm_display()

    def closeEvent(self, event):
        """Stop serial thread when window closes"""
        if self.serial_thread and self.serial_thread.isRunning():
            self.serial_thread.stop()
        event.accept()





    


# ---------------------- Program Entry ----------------------
if __name__ == '__main__':
    # High DPI adaptation
    QApplication.setAttribute(Qt.AA_EnableHighDpiScaling)
    QApplication.setAttribute(Qt.AA_UseHighDpiPixmaps)
    app = QApplication(sys.argv)
    window = TwoLinkArmController()
    window.show()
    sys.exit(app.exec_())

2026-01-06 16:48:03.586 python[22271:578482] error messaging the mach port for IMKCFRunLoopWakeUpReliable
2026-01-06 16:48:16.763 python[22271:578482] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
